In [7]:
! pip install implicit
! pip install ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.2/198.2 kB 745.6 kB/s eta 0:00:00a 0:00:01


In [25]:
import pandas as pd
from scipy import sparse
import implicit
from sklearn.preprocessing import MinMaxScaler

df = pd.read_csv("interactions.csv")
df_items = pd.read_csv("items.csv")

# drop columns which we don't need
df = df.drop(["TIMESTAMP", "DISCOUNT"], axis=1)

# add confidence scores
event_type_confidence = {
   "View": 1.0,
   "AddToCart": 2.0, 
   "ViewCart": 3.0, 
   "StartCheckout": 4.0,
   "Purchase": 5.0,  
}

df["CONFIDENCE"] = df["EVENT_TYPE"].apply(lambda x: event_type_confidence[x])
# 90,000 interactions
# print(df)

# this removes duplicates and adds up the confidence => down to 26K unique user-item interactions + confidence
grouped_df = df.groupby(["ITEM_ID", "USER_ID"]).sum("CONFIDENCE").reset_index()
grouped_df = grouped_df[["USER_ID", "ITEM_ID", "CONFIDENCE"]] # re-order columns
# print(grouped_df)

# train
grouped_df["USER_ID"] = grouped_df["USER_ID"].astype("category")
grouped_df["ITEM_ID"] = grouped_df["ITEM_ID"].astype("category")
print(f"Number of unique users: {grouped_df['USER_ID'].nunique()}")
print(f"Number of unique items: {grouped_df.ITEM_ID.nunique()}")
grouped_df["USER_IDX"] = grouped_df["USER_ID"].cat.codes
grouped_df["ITEM_IDX"] = grouped_df["ITEM_ID"].cat.codes
print(f"Min value user index: {grouped_df['USER_IDX'].min()}")
print(f"Max value user index: {grouped_df['USER_IDX'].max()}")
print(f"Min value item index: {grouped_df['ITEM_IDX'].min()}")
print(f"Max value item index: {grouped_df['ITEM_IDX'].max()}")
# print(grouped_df)

# Find items purchased by a given user
try_user = 168
try_result = grouped_df[grouped_df.USER_ID == try_user]
print("Try result:")
print(try_result)

sparse_person_content = sparse.csr_matrix((grouped_df["CONFIDENCE"].astype(float), (grouped_df["USER_IDX"], grouped_df["ITEM_IDX"])))
sparse_person_content

alpha = 15
model = implicit.als.AlternatingLeastSquares(alpha=alpha, factors=20, regularization=0.1, iterations=50)
user_vecs = model.user_factors
item_vecs = model.item_factors

data = (sparse_person_content).astype('double')

# # alpha = 15
# user_vecs, item_vecs = implicit.alternating_least_squares((product_train*alpha).astype('double'), 
#                                                           factors=20, 
#                                                           regularization = 0.1, 
#                                                          iterations = 50)


Number of unique users: 9587
Number of unique items: 2460
Min value user index: 0
Max value user index: 9586
Min value item index: 0
Max value item index: 2459
Try result:
      USER_ID                               ITEM_ID  CONFIDENCE  USER_IDX  \
12347     168  7220b5cd-110b-4105-813a-677846470816         3.0       158   
16727     168  9e676f4b-d23f-416e-9b12-ce2179c88caf         5.0       158   
22120     168  d16df944-ea75-43c3-b54b-6738731c081b         2.0       158   

       ITEM_IDX  
12347      1114  
16727      1544  
22120      2026  


In [27]:
model.fit(data)

  0%|          | 0/50 [00:00<?, ?it/s]

In [28]:
import random
import numpy as np

def make_train(ratings, pct_test=0.2):
    """
    This function will take in the original user-item matrix and "mask" a percentage of the original ratings where a
    user-item interaction has taken place for use as a test set. The test set will contain all of the original ratings,
    while the training set replaces the specified percentage of them with a zero in the original ratings matrix.

    parameters:

    ratings - the original ratings matrix from which you want to generate a train/test set. Test is just a complete
    copy of the original set. This is in the form of a sparse csr_matrix.

    pct_test - The percentage of user-item interactions where an interaction took place that you want to mask in the
    training set for later comparison to the test set, which contains all of the original ratings.

    returns:

    training_set - The altered version of the original data with a certain percentage of the user-item pairs
    that originally had interaction set back to zero.

    test_set - A copy of the original ratings matrix, unaltered, so it can be used to see how the rank order
    compares with the actual interactions.

    user_inds - From the randomly selected user-item indices, which user rows were altered in the training data.
    This will be necessary later when evaluating the performance via AUC.
    """
    test_set = ratings.copy()  # Make a copy of the original set to be the test set.
    test_set[test_set != 0] = 1  # Store the test set as a binary preference matrix
    training_set = (
        ratings.copy()
    )  # Make a copy of the original data we can alter as our training set.
    nonzero_inds = (
        training_set.nonzero()
    )  # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(
        zip(nonzero_inds[0], nonzero_inds[1])
    )  # Zip these pairs together of user,item index into list
    random.seed(0)  # Set the random seed to zero for reproducibility
    num_samples = int(
        np.ceil(pct_test * len(nonzero_pairs))
    )  # Round the number of samples needed to the nearest integer
    samples = random.sample(
        nonzero_pairs, num_samples
    )  # Sample a random number of user-item pairs without replacement
    user_inds = [index[0] for index in samples]  # Get the user row indices
    item_inds = [index[1] for index in samples]  # Get the item column indices
    training_set[
        user_inds, item_inds
    ] = 0  # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros()  # Get rid of zeros in sparse array storage after update to save space
    return (
        training_set,
        test_set,
        list(set(user_inds)),
    )  # Output the unique list of user rows that were altered
    
product_train, product_test, product_users_altered = make_train(sparse_person_content)
print(product_train)
print("------")
print(product_test)
# print(product_users_altered)
    

  (0, 2007)	4.0
  (1, 483)	4.0
  (1, 1748)	6.0
  (2, 530)	3.0
  (2, 927)	6.0
  (3, 154)	7.0
  (3, 604)	3.0
  (4, 275)	3.0
  (5, 614)	4.0
  (5, 761)	2.0
  (6, 183)	2.0
  (6, 1869)	18.0
  (7, 1506)	2.0
  (7, 2322)	3.0
  (8, 2139)	9.0
  (9, 345)	4.0
  (9, 602)	2.0
  (9, 2104)	2.0
  (10, 422)	3.0
  (10, 529)	3.0
  (10, 974)	2.0
  (10, 1119)	5.0
  (10, 1638)	3.0
  (12, 602)	2.0
  (12, 803)	2.0
  :	:
  (9567, 1708)	2.0
  (9568, 126)	8.0
  (9570, 87)	6.0
  (9571, 87)	2.0
  (9571, 2170)	2.0
  (9572, 70)	2.0
  (9573, 87)	10.0
  (9574, 1060)	2.0
  (9574, 1725)	1.0
  (9575, 70)	1.0
  (9576, 1191)	6.0
  (9577, 346)	8.0
  (9577, 354)	4.0
  (9577, 842)	2.0
  (9577, 885)	2.0
  (9578, 842)	3.0
  (9579, 126)	2.0
  (9580, 885)	2.0
  (9581, 1725)	5.0
  (9582, 87)	1.0
  (9583, 70)	1.0
  (9583, 1708)	2.0
  (9584, 1060)	2.0
  (9585, 87)	4.0
  (9586, 1191)	2.0
------
  (0, 2007)	1.0
  (1, 483)	1.0
  (1, 1483)	1.0
  (1, 1748)	1.0
  (2, 120)	1.0
  (2, 530)	1.0
  (2, 927)	1.0
  (2, 1490)	1.0
  (2, 2311)	1.0
  (

In [29]:
from sklearn import metrics

def auc_score(predictions, test):
    '''
    This simple function will output the area under the curve using sklearn's metrics. 
    
    parameters:
    
    - predictions: your prediction output
    
    - test: the actual target result you are comparing to
    
    returns:
    
    - AUC (area under the Receiver Operating Characterisic curve)
    '''
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr)

In [30]:
def calc_mean_auc(training_set, altered_users, predictions, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    
    
    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    popularity_auc = [] # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # Get sum of item iteractions to find most popular
    item_vecs = predictions[1]
    for user in altered_users: # Iterate through each user that had an item altered
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
    # End users iteration
    
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark

In [31]:
calc_mean_auc(product_train, product_users_altered, 
              [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
# AUC for our recommender system

ValueError: object dtype is not supported by sparse matrices

In [17]:
# Get popular items (to use as baseline)
popular_items = grouped_df.ITEM_ID.value_counts(sort=True).keys()[:10]
for item in popular_items:
    item_desc = df_items.PRODUCT_DESCRIPTION.loc[df_items.ITEM_ID == item].iloc[0]
    print(f"Item ID: {item}, Desc: {item_desc}")

Item ID: 0790267c-c708-424d-81f5-46903a9c8444, Desc: Slice of delicious pepperoni pizza
Item ID: b20ba076-58a7-4602-9b56-4bee46e98388, Desc: The best nachos north of Mexico
Item ID: 575c0ac0-5494-4c64-a886-a9c0cf8b779a, Desc: Lentils with potatos, carrots and spices; a joy to your palate and super healthy
Item ID: 0987bfa1-0a23-4b90-8882-8a6e9bd91e24, Desc: Juicy prawns with spicy sauce and rice
Item ID: aff05423-76e8-4339-a478-fc17d51ed985, Desc: 16oz fountain soda always hits the spot
Item ID: 25d7bbf6-7dd3-4912-93a7-4186ea417b54, Desc: Full of omega oils and taste
Item ID: 9c1a2048-7aac-4565-b836-d8d4f726322c, Desc: Crunchy and yummy! Crumbs in your lap!
Item ID: 24c62ad2-6977-4f69-be75-e37d897c1434, Desc: Warms you to your bowels and delivers vitamins to your organs
Item ID: a6f43f84-a89a-446f-8adc-8b1a23a30a81, Desc: Exotic and healthy, grape-vine leaves stuffed with rice... and spice
Item ID: 5afced84-ed2d-4520-a06d-dcfeab382e52, Desc: Made with ginseng and joy


In [18]:
user_idx = 158
test = model.recommend(user_idx, sparse_person_content[user_idx])

print(test)

print(f"User idx: {user_idx}")
for idx in range(0, 10):
    item_id = grouped_df.ITEM_ID.loc[grouped_df.ITEM_IDX == test[0][idx]].iloc[0]
    item_desc = df_items.PRODUCT_DESCRIPTION.loc[df_items.ITEM_ID == item_id].iloc[0]
    # {test[0][idx]}, \
    print(f" \
          {item_id}, \
          {test[1][idx]}, \
          {item_desc}")
    


(array([1283, 1433,  244, 1696, 1882,  894, 1133, 2448,   87,  802],
      dtype=int32), array([0.49735963, 0.49549776, 0.46319824, 0.41714284, 0.40533042,
       0.40176946, 0.39922363, 0.39818406, 0.39794934, 0.39234784],
      dtype=float32))
User idx: 158
           844846d9-dc61-4709-8286-a0f79d6ae809,           0.49735963344573975,           Fashionable glasses for women
           92caea3c-23c8-4bb0-8a4b-43372adafaae,           0.49549776315689087,           Dark salmon handbag for those dressy occasions
           1a3fa9d4-e320-4873-be58-f3f6af5f99f4,           0.4631982445716858,           Rust leather backpack for women
           aedc65f7-684c-4e4a-bcd0-455dae355ced,           0.4171428382396698,           This hip rust leather handbag is the perfect accessory for casually wandering everywhere
           c15d7a38-9df9-44ee-8733-6bda17f89e8c,           0.4053304195404053,           Linen hue handbag for those dressy occasions
           5c010bac-31a6-4ff3-9557-2f5e20b509cc,  

In [1]:
# https://nbviewer.org/github/jmsteinw/Notebooks/blob/master/RecEngine_NB.ipynb
# This notebook shows awesome extensive process for implicit feedback systems. 